In [1]:
FILE_HEADER = 'SDATA version 2.0'
SEGMENT_HEADER = '2 2 2 : NX NY NT'
empty_line = ''

# CELL HEADER
NPIXELS = 0
TIMESTAMP = '2008-06-14T14:49:28Z'
HEIGHT_OBS = 70000
NSURF = 0
IFGAS = 0

# The PIXEL structure
ix = 1
iy = 1
icloudy = 1

icol = 1360
irow = 1360

x = -17.0130005
y = 14.4720001 
MASL = 57.0000000  
land_percent = 0.00000000 

# number of available wavelengths 
nwl = 6 
# list of wavelengths, in micrometers
wl = [0.442999989, 0.490000010, 0.564999998, 0.670000017, 0.865000010, 1.01999998] 
# number of types of measurements for each wavelength 
nip = [1, 3, 1, 3, 3, 1]
# list of types of measurements, meas_type(nip, wl)
meas_type = []
# number of valid measurements 
nbvm = []

# solar zenith angle, for each wavelength
sza = []
# viewing zenith angle, θv(nbvm, nip, wl)
vza = []
# relative azimuth angle, Δϕ(nbvm, nip, wln)
raa = []

# measurements, meas(nbvm, nip, wln)
meas = []

# 1 if a covariance matrix is available, 0 otherwise, ifcov(nip, wln)
ifcov = []

# 1 if a vertical profile (mprof) is available, 0 otherwise, ifmp(nip, wln)
ifmp = []

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('example_polder_normalize_data.csv')

In [13]:
timestamp_table = df.groupby('TIMESTAMP')['HEIGHT_OBS', 'NPIXELS', 'NSURF', 'IFGAS'].max()

In [229]:
import json
with open('timestamp_info.json', 'w') as file:
    json.dump(timestamp_dict, file)

In [216]:
timestamp_dict = timestamp_table.T.to_dict()

In [32]:
pixel_table = df.groupby(['ix', 'iy'])['icloudy', 'icol', 'irow', 'x', 'y', 'MASL', 'land_percent','nwl'].min()

In [215]:
pixel_dict = pixel_table.T.to_dict()

In [45]:
t = df.groupby(['ix', 'iy', 'wl']).nip.mean().rename('nip').reset_index()
pixel_table_2 = pd.concat([t.groupby(['ix', 'iy'])['wl'].apply(list), t.groupby(['ix', 'iy'])['nip'].apply(list)], axis=1)

In [97]:
df.groupby(['TIMESTAMP','ix', 'iy', 'wl']).nbvm.apply(pd.Series.nunique)

TIMESTAMP             ix  iy  wl   
2008-06-14T14:49:28Z  1   1   0.443    1
                              0.490    1
                              0.565    1
                              0.670    1
                              0.865    1
                              1.020    1
                          2   0.443    1
                              0.490    1
                              0.565    1
                              0.670    1
                              0.865    1
                              1.020    1
                      2   1   0.443    1
                              0.490    1
                              0.565    1
                              0.670    1
                              0.865    1
                              1.020    1
                          2   0.443    1
                              0.490    1
                              0.565    1
                              0.670    1
                              0.865    1
                     

In [200]:
par_list = ['TIMESTAMP','ix', 'iy', 'wl','nip', 'sza', 'meas_type', 'ifcov', 'ifmp']


vza_values = df.groupby(par_list).vza.apply(list).rename('vza')
raa_values = df.groupby(par_list).raa.apply(list).rename('raa')
meas_values = df.groupby(par_list).meas.apply(list).rename('meas')

In [236]:
dict_of_dicts = {}

table = pd.concat([vza_values, raa_values, meas_values], axis=1).reset_index()
for timestamp in timestamp_dict:
    for ix in range(2):
        for iy in range(2):
            
            r = table[(table['TIMESTAMP'] == '2008-06-14T14:49:28Z') & (table['ix'] == ix + 1) & (table['iy'] == iy + 1)]
            r = r.set_index(par_list)
            final_dict = {}
            
            final_dict['vza'] = r['vza'].sum()
            final_dict['raa'] = r['raa'].sum()
            final_dict['meas'] = r['meas'].sum()

            r_reset = r.reset_index().drop(['vza','raa','meas'], 1)
            final_dict['meas_type'] = r_reset['meas_type'].tolist()
            final_dict['ifcov'] = r_reset['ifcov'].tolist()
            final_dict['ifmp'] = r_reset['ifmp'].tolist()

            wl_gr = r_reset.groupby(['TIMESTAMP','ix', 'iy', 'wl'])

            final_dict['nip'] = wl_gr.meas_type.nunique().tolist()

            wl_gr_df = wl_gr.max().reset_index()

            final_dict['sza'] = wl_gr_df['sza'].tolist()
            final_dict['nip'] = wl_gr_df['nip'].tolist()
            final_dict['wl'] = wl_gr_df['wl'].tolist()

            final_dict.update(pixel_dict[(ix + 1, iy + 1)])
            
            dict_of_dicts[(timestamp, ix, iy)] = final_dict

In [238]:
dict_of_dicts[('2008-06-14T14:49:28Z',
  0,
  0)]

{'vza': [58.837101000000004,
  54.179599800000005,
  48.8362999,
  42.8232994,
  36.417301200000004,
  30.431999199999996,
  26.581899600000003,
  26.773000699999997,
  30.896799100000003,
  36.969299299999996,
  43.3571014,
  49.3139,
  54.5943985,
  59.1898003,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.70

In [223]:
final_dict

{'vza': [58.837101000000004,
  54.179599800000005,
  48.8362999,
  42.8232994,
  36.417301200000004,
  30.431999199999996,
  26.581899600000003,
  26.773000699999997,
  30.896799100000003,
  36.969299299999996,
  43.3571014,
  49.3139,
  54.5943985,
  59.1898003,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.70

In [224]:
r_reset['meas_type']

0     41
1     41
2     42
3     43
4     41
5     41
6     42
7     43
8     41
9     42
10    43
11    41
Name: meas_type, dtype: int64

In [225]:
final_dict

{'vza': [58.837101000000004,
  54.179599800000005,
  48.8362999,
  42.8232994,
  36.417301200000004,
  30.431999199999996,
  26.581899600000003,
  26.773000699999997,
  30.896799100000003,
  36.969299299999996,
  43.3571014,
  49.3139,
  54.5943985,
  59.1898003,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.973701500000004,
  54.3372002,
  49.0166016,
  43.0206985,
  36.619201700000005,
  30.5984001,
  26.649299600000003,
  26.700199100000003,
  30.729799300000003,
  36.7719994,
  43.1637001,
  49.1403999,
  54.4426003,
  59.05559920000001,
  58.70

In [174]:
r_reset

,TIMESTAMP,ix,iy,wl,nip,sza,meas_type
0,2008-06-14T14:49:28Z,1,1,0.443,1,24.109501,41
1,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,41
2,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,42
3,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,43
4,2008-06-14T14:49:28Z,1,1,0.565,1,24.109501,41
5,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,41
6,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,42
7,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,43
8,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,41
9,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,42


nip        sza  meas_type
TIMESTAMP            ix iy wl                              
2008-06-14T14:49:28Z 1  1  0.443    1  24.109501         41
                           0.490    3  24.109501         43
                           0.565    1  24.109501         41
                           0.670    3  24.109501         43
                           0.865    3  24.109501         43
                           1.020    1  24.109501         41

In [162]:
r.reset_index()

,TIMESTAMP,ix,iy,wl,nip,sza,meas_type,vza,raa,meas
0,2008-06-14T14:49:28Z,1,1,0.443,1,24.109501,41,"[58.837101000000004, 54.179599800000005, 48.83...","[107.692001, 104.415001, 99.9546967, 93.609298...","[0.18758133100000002, 0.17888978100000003, 0.1..."
1,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,41,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[0.167410657, 0.15861778, 0.149844468, 0.14162..."
2,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,42,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[-0.041035451, -0.0329722129, -0.0242278314999..."
3,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,43,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[-0.046323519199999996, -0.0409283862, -0.0348..."
4,2008-06-14T14:49:28Z,1,1,0.565,1,24.109501,41,"[58.70069889999999, 54.0222015, 48.6565018, 42...","[107.602997, 104.29699699999999, 99.7929001, 9...","[0.141543269, 0.133097783, 0.12492681300000001..."
5,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,41,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[0.109973587, 0.102243394, 0.09498795119999999..."
6,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,42,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[-0.0306082368, -0.0240230057, -0.017220281100..."
7,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,43,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[-0.0324008092, -0.0278445352, -0.0229491219, ..."
8,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,41,"[58.156601, 53.394901299999994, 47.94100189999...","[107.244003, 103.817001, 99.1339035, 92.399299...","[0.07350265980000001, 0.0676775202, 0.06233778..."
9,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,42,"[58.156601, 53.394901299999994, 47.94100189999...","[107.244003, 103.817001, 99.1339035, 92.399299...","[-0.0225871634, -0.0173334051, -0.0120806126, ..."


In [135]:
r.T

TIMESTAMP                               2008-06-14T14:49:28Z  \
ix                                                         1   
iy                                                         1   
wl                                                     0.443   
nip                                                        1   
sza                                                24.109501   
meas_type                                                 41   
vza        [58.837101000000004, 54.179599800000005, 48.83...   
raa        [107.692001, 104.415001, 99.9546967, 93.609298...   
meas       [0.18758133100000002, 0.17888978100000003, 0.1...   

TIMESTAMP                                                     \
ix                                                             
iy                                                             
wl                                                     0.490   
nip                                                        3   
sza                                                24.109501   
meas_type                                                 41   
vza        [58.973701500000004, 54.3372002, 49.0166016, 4...   
raa        [107.779999, 104.532997, 100.114998, 93.844703...   
meas       [0.167410657, 0.15861778, 0.149844468, 0.14162...   

TIMESTAMP                                                     \
ix                                                             
iy                                                             
wl                                                             
nip                                                            
sza                                                            
meas_type                                                 42   
vza        [58.973701500000004, 54.3372002, 49.0166016, 4...   
raa        [107.779999, 104.532997, 100.114998, 93.844703...   
meas       [-0.041035451, -0.0329722129, -0.0242278314999...   

TIMESTAMP                                                     \
ix                                                             
iy                                                             
wl                                                             
nip                                                            
sza                                                            
meas_type                                                 43   
vza        [58.973701500000004, 54.3372002, 49.0166016, 4...   
raa        [107.779999, 104.532997, 100.114998, 93.844703...   
meas       [-0.046323519199999996, -0.0409283862, -0.0348...   

TIMESTAMP                                                     \
ix                                                             
iy                                                             
wl                                                     0.565   
nip                                                        1   
sza                                                24.109501   
meas_type                                                 41   
vza        [58.70069889999999, 54.0222015, 48.6565018, 42...   
raa        [107.602997, 104.29699699999999, 99.7929001, 9...   
meas       [0.141543269, 0.133097783, 0.12492681300000001...   

TIMESTAMP                                                     \
ix                                                             
iy                                                             
wl                                                     0.670   
nip                                                        3   
sza                                                24.109501   
meas_type                                                 41   
vza        [58.5644989, 53.8650017, 48.477001200000004, 4...   
raa        [107.514, 104.178001, 99.6299973, 93.132003799...   
meas       [0.109973587, 0.102243394, 0.09498795119999999...   

TIMESTAMP                                                     \
ix                                                             
iy                        

,TIMESTAMP,ix,iy,wl,nip,sza,meas_type,vza,raa,meas
0,2008-06-14T14:49:28Z,1,1,0.443,1,24.109501,41,"[58.837101000000004, 54.179599800000005, 48.83...","[107.692001, 104.415001, 99.9546967, 93.609298...","[0.18758133100000002, 0.17888978100000003, 0.1..."
1,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,41,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[0.167410657, 0.15861778, 0.149844468, 0.14162..."
2,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,42,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[-0.041035451, -0.0329722129, -0.0242278314999..."
3,2008-06-14T14:49:28Z,1,1,0.490,3,24.109501,43,"[58.973701500000004, 54.3372002, 49.0166016, 4...","[107.779999, 104.532997, 100.114998, 93.844703...","[-0.046323519199999996, -0.0409283862, -0.0348..."
4,2008-06-14T14:49:28Z,1,1,0.565,1,24.109501,41,"[58.70069889999999, 54.0222015, 48.6565018, 42...","[107.602997, 104.29699699999999, 99.7929001, 9...","[0.141543269, 0.133097783, 0.12492681300000001..."
5,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,41,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[0.109973587, 0.102243394, 0.09498795119999999..."
6,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,42,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[-0.0306082368, -0.0240230057, -0.017220281100..."
7,2008-06-14T14:49:28Z,1,1,0.670,3,24.109501,43,"[58.5644989, 53.8650017, 48.477001200000004, 4...","[107.514, 104.178001, 99.6299973, 93.132003799...","[-0.0324008092, -0.0278445352, -0.0229491219, ..."
8,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,41,"[58.156601, 53.394901299999994, 47.94100189999...","[107.244003, 103.817001, 99.1339035, 92.399299...","[0.07350265980000001, 0.0676775202, 0.06233778..."
9,2008-06-14T14:49:28Z,1,1,0.865,3,24.109501,42,"[58.156601, 53.394901299999994, 47.94100189999...","[107.244003, 103.817001, 99.1339035, 92.399299...","[-0.0225871634, -0.0173334051, -0.0120806126, ..."


In [46]:
pixel_table.join(pixel_table_2)

icloudy  icol  irow          x       y  MASL  land_percent  nwl  \
ix iy                                                                    
1  1         1  1360  2944 -17.013001  14.472  57.0           0.0    6   
   2         1  1361  2944 -17.007999  14.417  43.0          50.0    6   
2  1         1  1360  2945 -16.955999  14.472  46.0         100.0    6   
   2         1  1361  2945 -16.950001  14.417  38.0         100.0    6   

                                                      wl                 nip  
ix iy                                                                         
1  1   [0.442999989, 0.49000001, 0.564999998, 0.67000...  [1, 3, 1, 3, 3, 1]  
   2   [0.442999989, 0.49000001, 0.564999998, 0.67000...  [1, 3, 1, 3, 3, 1]  
2  1   [0.442999989, 0.49000001, 0.564999998, 0.67000...  [1, 3, 1, 3, 3, 1]  
   2   [0.442999989, 0.49000001, 0.564999998, 0.67000...  [1, 3, 1, 3, 3, 1]

In [31]:
df.groupby(['TIMESTAMP', 'ix', 'iy', 'nwl', 'wl']).meas_type.apply(list)

TIMESTAMP             ix  iy  nwl  wl   
2008-06-14T14:49:28Z  1   1   6    0.443    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.490    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.565    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.670    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.865    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   1.020    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                          2   6    0.443    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.490    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.565    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                                   0.670    [41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 4...
                   

In [24]:
df

,ix,iy,icloudy,icol,irow,x,y,MASL,land_percent,nwl,...,vza,raa,meas,ifcov,ifmp,NPIXELS,TIMESTAMP,HEIGHT_OBS,NSURF,IFGAS
0,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,58.837101,107.692001,0.187581,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
1,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,54.179600,104.415001,0.178890,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
2,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,48.836300,99.954697,0.170240,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
3,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,42.823299,93.609299,0.162257,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
4,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,36.417301,84.111504,0.155974,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
5,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,30.431999,69.401802,0.154594,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
6,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,26.581900,47.794300,0.155746,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
7,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,26.773001,22.533701,0.159671,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
8,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,30.896799,1.505100,0.164776,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0
9,1,1,1,1360,2944,-17.013001,14.472,57.0,0.0,6,...,36.969299,347.343994,0.169371,0,0,4,2008-06-14T14:49:28Z,705000.0,0,0


In [ ]:
data.groupby(['TIMESTAMP']).apply(lambda x: x.to_json(orient='records'))

In [5]:
df.groupby('TIMESTAMP').raa.apply(list)

TIMESTAMP
2008-06-14T14:49:28Z    [107.692001, 104.415001, 99.9546967, 93.609298...
2008-06-16T14:37:07Z    [131.220001, 131.33500700000002, 131.518996999...
2008-06-18T14:24:47Z    [155.744003, 159.253006, 164.067993, 170.92199...
2008-06-20T14:12:28Z    [174.804993, 179.048004, 184.272995, 190.71099...
2008-06-23T14:43:18Z    [120.245003, 118.704002, 116.574997, 113.445, ...
2008-06-25T14:30:56Z    [144.40600600000002, 146.546997, 149.671997, 1...
2008-06-27T14:18:35Z    [165.188004, 169.024002, 173.951996, 180.37399...
2008-06-30T14:49:28Z    [109.070999, 106.153999, 102.245003, 96.765899...
2008-07-02T14:37:06Z    [131.25100700000002, 131.434006, 131.746002, 1...
2008-07-04T14:24:43Z    [155.664001, 159.363998, 164.48100300000002, 1...
2008-07-06T14:12:23Z    [173.65600600000002, 177.91099499999999, 183.1...
2008-07-09T14:43:18Z    [117.960999, 116.434998, 114.346001, 111.25900...
2008-07-11T14:30:56Z    [140.615005, 142.509003, 145.210999, 149.31100...
2008-07-13T14:18:34Z    [162

In [118]:
# grouped = df.groupby(['TIMESTAMP', 'ix']).apply(list).to_frame()
grouped = r.copy()['vza'].to_frame()

In [119]:
grouped

vza
TIMESTAMP            ix iy wl    nip sza       meas_type                                                   
2008-06-14T14:49:28Z 1  1  0.443 1   24.109501 41         [58.837101000000004, 54.179599800000005, 48.83...
                           0.490 3   24.109501 41         [58.973701500000004, 54.3372002, 49.0166016, 4...
                                               42         [58.973701500000004, 54.3372002, 49.0166016, 4...
                                               43         [58.973701500000004, 54.3372002, 49.0166016, 4...
                           0.565 1   24.109501 41         [58.70069889999999, 54.0222015, 48.6565018, 42...
                           0.670 3   24.109501 41         [58.5644989, 53.8650017, 48.477001200000004, 4...
                                               42         [58.5644989, 53.8650017, 48.477001200000004, 4...
                                               43         [58.5644989, 53.8650017, 48.477001200000004, 4...
                           0.865 3   24.109501 41         [58.156601, 53.394901299999994, 47.94100189999...
                                               42         [58.156601, 53.394901299999994, 47.94100189999...
                                               43         [58.156601, 53.394901299999994, 47.94100189999...
                           1.020 1   24.109501 41         [58.768901799999995, 54.1007996, 48.7462997, 4...
                        2  0.443 1   24.138001 41         [58.63320160000001, 53.94250110000001, 48.5640...
                           0.490 3   24.138001 41         [58.769699100000004, 54.1001015, 48.7447014, 4...
                                               42         [58.769699100000004, 54.1001015, 48.7447014, 4...
                                               43         [58.769699100000004, 54.1001015, 48.7447014, 4...
                           0.565 1   24.138001 41         [58.4967995, 53.785099, 48.3838005, 42.3247986...
                           0.670 3   24.138001 41         [58.360500300000005, 53.627998399999996, 48.20...
                                               42         [58.360500300000005, 53.627998399999996, 48.20...
                                               43         [58.360500300000005, 53.627998399999996, 48.20...
                           0.865 3   24.138001 41         [57.952499399999994, 53.1580009, 47.667099, 41...
                                               42         [57.952499399999994, 53.1580009, 47.667099, 41...
                                               43         [57.952499399999994, 53.1580009, 47.667099, 41...
                           1.020 1   24.138001 41         [58.5649986, 53.8638, 48.4738998, 42.424499499...
                     2  1  0.443 1   24.160500 41         [58.864299800000005, 54.224201199999996, 48.91...
                           0.490 3   24.160500 41         [59.00009920000001, 54.38090129999999, 49.0906...
                                               42         [59.00009920000001, 54.38090129999999, 49.0906...
                                               43         [59.00009920000001, 54.38090129999999, 49.0906...
                           0.565 1   24.160500 41         [58.7285995, 54.067699399999995, 48.7321014, 4...
                           0.670 3   24.160500 41         [58.5929985, 53.911499, 48.5535011, 42.5579987...
...                                                                                                     ...
2008-08-19T14:36:24Z 1  2  0.670 3   19.663500 42         [58.57799910000001, 53.51549910000001, 47.4599...
                                               43         [58.57799910000001, 53.51549910000001, 47.4599...
                           0.865 3   19.663500 41         [58.1409988, 53.00120160000001, 46.838501, 39....
                                               42         [58.1409988, 53.00120160000001, 46.838501, 39....
                                               43         [58.1409988, 53.00120160000001, 46.838501, 39....


In [120]:
levels = len(grouped.index.levels)
dicts = [{} for i in range(levels)]
last_index = None

for index,value in grouped.itertuples():

    if not last_index:
        last_index = index

    for (ii,(i,j)) in enumerate(zip(index, last_index)):
        if not i == j:
            ii = levels - ii -1
            dicts[:ii] =  [{} for _ in dicts[:ii]]
            break

    for i, key in enumerate(reversed(index)):
        dicts[i][key] = value
        value = dicts[i]

    last_index = index

In [121]:
dicts[-1]['2008-06-14T14:49:28Z'][1][1][0.442999989]

{1: {24.1095009: {41: [58.837101000000004,
    54.179599800000005,
    48.8362999,
    42.8232994,
    36.417301200000004,
    30.431999199999996,
    26.581899600000003,
    26.773000699999997,
    30.896799100000003,
    36.969299299999996,
    43.3571014,
    49.3139,
    54.5943985,
    59.1898003]}}}

In [93]:
import json
json.dumps(dicts[-1])

'{"2008-06-14T14:49:28Z": {"1": {"1": {"0.442999989": {"1": {"24.1095009": {"41": [58.837101000000004, 54.179599800000005, 48.8362999, 42.8232994, 36.417301200000004, 30.431999199999996, 26.581899600000003, 26.773000699999997, 30.896799100000003, 36.969299299999996, 43.3571014, 49.3139, 54.5943985, 59.1898003]}}}, "0.49000001": {"3": {"24.1095009": {"41": [58.973701500000004, 54.3372002, 49.0166016, 43.0206985, 36.619201700000005, 30.5984001, 26.649299600000003, 26.700199100000003, 30.729799300000003, 36.7719994, 43.1637001, 49.1403999, 54.4426003, 59.05559920000001], "42": [58.973701500000004, 54.3372002, 49.0166016, 43.0206985, 36.619201700000005, 30.5984001, 26.649299600000003, 26.700199100000003, 30.729799300000003, 36.7719994, 43.1637001, 49.1403999, 54.4426003, 59.05559920000001], "43": [58.973701500000004, 54.3372002, 49.0166016, 43.0206985, 36.619201700000005, 30.5984001, 26.649299600000003, 26.700199100000003, 30.729799300000003, 36.7719994, 43.1637001, 49.1403999, 54.4426003,

In [75]:
from pprint import pprint
print(dict(results['2008-06-14T14:49:28Z']))

{1: {1: ['ix', 'iy', 'icloudy', 'icol', 'irow', 'x', 'y', 'MASL', 'land_percent', 'nwl', 'wl', 'nip', 'meas_type', 'nbvm', 'sza', 'vza', 'raa', 'meas', 'ifcov', 'ifmp', 'NPIXELS', 'TIMESTAMP', 'HEIGHT_OBS', 'NSURF', 'IFGAS'], 2: ['ix', 'iy', 'icloudy', 'icol', 'irow', 'x', 'y', 'MASL', 'land_percent', 'nwl', 'wl', 'nip', 'meas_type', 'nbvm', 'sza', 'vza', 'raa', 'meas', 'ifcov', 'ifmp', 'NPIXELS', 'TIMESTAMP', 'HEIGHT_OBS', 'NSURF', 'IFGAS']}, 2: {1: ['ix', 'iy', 'icloudy', 'icol', 'irow', 'x', 'y', 'MASL', 'land_percent', 'nwl', 'wl', 'nip', 'meas_type', 'nbvm', 'sza', 'vza', 'raa', 'meas', 'ifcov', 'ifmp', 'NPIXELS', 'TIMESTAMP', 'HEIGHT_OBS', 'NSURF', 'IFGAS'], 2: ['ix', 'iy', 'icloudy', 'icol', 'irow', 'x', 'y', 'MASL', 'land_percent', 'nwl', 'wl', 'nip', 'meas_type', 'nbvm', 'sza', 'vza', 'raa', 'meas', 'ifcov', 'ifmp', 'NPIXELS', 'TIMESTAMP', 'HEIGHT_OBS', 'NSURF', 'IFGAS']}}


In [ ]:
data[(data['ix'] == '1') & (data['iy'] == '1') & (data.TIMESTAMP == '2008-06-14T14:49:28Z')].nunique()